In [2]:
import os
import sys 
import numpy as np 
import tensorflow as tf
import operator
batch_size =50 

def load_model(session, save_path):
    """ Loads a saved TF model from a file.Returns:The inputs placehoder. """
    print("Loading model from file '%s'..." % (save_path))
    meta_file = save_path + ".meta"
    if not os.path.exists(meta_file):
        print("ERROR: Expected .meta file '%s', but could not find it." % \
        (meta_file))
        sys.exit(1) 
    saver = tf.train.import_meta_graph(meta_file)
    save_path = os.path.join("./", save_path)
    saver.restore(session, save_path)  
    return extract_validation_handles(session)
    
def load_validation_data(model, testname):
    global batch_size 
    """ Loads the validation data,  Returns: A tuple of the loaded validation data and validation labels. """
    print("Loading validation data...")
    folder = 'time_series_data/' + testname 
    if model ==1:
        testX =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/'+folder+'/testX_pred.npy' ) 
        testY =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/'+folder+'/testY_pred.npy'  )  
    elif model==2:
        testX =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/'+folder+'/testX_no.npy' ) 
        testY =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/'+folder+'/testY_no.npy' )  
    if testX.shape[1]!= 214:
        testX = np.transpose(testX,[0,2,1])
        testX = np.expand_dims(testX,3) 
    batch_size =np.shape(testX)[0]   
    return (testX,testY )  

def validate_model(session, val_data, x, y,  phase_train,prob): 
    """ Validates the model stored in a session.Returns:The overall validation accuracy for the model. """
    global batch_size
    print("Validating model...") 
    predict_op = tf.argmax(prob, 1)
    correct = tf.equal(predict_op,tf.argmax(y, 1))
    acc= tf.reduce_mean(tf.cast(correct, tf.float32))  
    val_x, val_y  = val_data
    test_correct = session.run(correct,feed_dict ={x: val_x,  y:val_y, phase_train: True})
    test_acc = session.run(acc,feed_dict ={x: val_x, y:val_y, phase_train: True})
    return ( test_acc), test_correct  

def extract_validation_handles(session):
    """ Extracts the input that we use for validation.
    Args:
      session: The session with the loaded graph.
    Returns:
      The inputs placeholders and probability """ 
    valid_nodes = tf.get_collection_ref("validation_nodes") 
    x = valid_nodes[0] 
    y = valid_nodes[1] 
    phase_train =valid_nodes[2]
    prob =  valid_nodes[3]
    global batch_size    
    return (x,y,  phase_train,prob)

def each_perform(correct_results,eval_labels ):
    label_y=np.argmax(eval_labels,1) 
    
    acc = []
    zero = np.where(label_y==0) 
    correctl=[correct_results[i] for i in zero]
    accuracy_zero= np.mean(correctl,dtype=np.float32)
    acc.append(accuracy_zero)

    one = np.where(label_y==1)
    correctg=[correct_results[i] for i in one] 
    accuracy_one= np.mean(correctg,dtype=np.float32)
    acc.append(accuracy_one)

    two = np.where(label_y==2)
    correctt=[correct_results[i] for i in two] 
    accuracy_two= np.mean(correctt,dtype=np.float32) 
    acc.append(accuracy_two) 
    return acc, correctl,correctg,correctt

def reset():
    tf.reset_default_graph()

model = 1
testname = 'total'  
names = 'modelC_fully02' 
with tf.Session() as session:  
    save_path = 'E:/05_research/04_deep_learning_model/Save_parameters/modelC/'+ names 
    x, y,phase_train,prob  = load_model(session, save_path)
    val_data = load_validation_data(model, testname) # the predicted mode
    accuracy ,correct_results= validate_model(session, val_data, x,y, phase_train,prob)
    print ("Overall validation accuracy is %f \n" %(accuracy))
    acc,correct_l,correct_g,correct_t= each_perform(correct_results,val_data[1] ) 
    session.close() 
    #reset() 

print (100*np.mean(correct_l[0] ))
print (100*np.mean(correct_g[0]))
print (100*np.mean(correct_t[0] ))
 

Loading model from file 'E:/05_research/04_deep_learning_model/Save_parameters/modelC/modelC_fully02'...
INFO:tensorflow:Restoring parameters from E:/05_research/04_deep_learning_model/Save_parameters/modelC/modelC_fully02
Loading validation data...
Validating model...
Overall validation accuracy is 0.704437 

86.24338624338624
54.976303317535546
71.64502164502164


In [3]:
id_false =  np.nonzero(correct_results==False) 
id_false

(array([   0,    1,    2,    7,    8,   13,   19,   20,   25,   26,   27,
          32,   33,   38,   39,   40,   45,   46,   51,   52,   53,   58,
          59,   64,   65,   66,   71,   72,   90,   91,   92,   95,   97,
          98,  103,  104,  105,  109,  111,  112,  117,  118,  119,  120,
         124,  125,  130,  131,  132,  133,  137,  138,  143,  144,  145,
         146,  150,  151,  152,  156,  157,  158,  163,  164,  169,  170,
         171,  172,  175,  176,  177,  178,  188,  196,  197,  198,  199,
         200,  201,  202,  204,  205,  206,  215,  216,  218,  226,  227,
         229,  230,  231,  232,  233,  234,  236,  237,  238,  239,  240,
         244,  245,  247,  251,  252,  253,  256,  257,  258,  260,  263,
         272,  274,  275,  341,  342,  343,  344,  345,  346,  347,  349,
         350,  351,  352,  353,  370,  371,  372,  373,  374,  375,  376,
         378,  380,  390,  398,  399,  400,  401,  402,  403,  404,  407,
         409,  410,  419,  427,  428, 

In [4]:
id_f = [   0,    1,    2,    7,    8,   13,   19,   20,   25,   26,   27,
          32,   33,   38,   39,   40,   45,   46,   51,   52,   53,   58,
          59,   64,   65,   66,   71,   72,   90,   91,   92,   95,   97,
          98,  103,  104,  105,  109,  111,  112,  117,  118,  119,  120,
         124,  125,  130,  131,  132,  133,  137,  138,  143,  144,  145,
         146,  150,  151,  152,  156,  157,  158,  163,  164,  169,  170,
         171,  172,  175,  176,  177,  178,  188,  196,  197,  198,  199,
         200,  201,  202,  204,  205,  206,  215,  216,  218,  226,  227,
         229,  230,  231,  232,  233,  234,  236,  237,  238,  239,  240,
         244,  245,  247,  251,  252,  253,  256,  257,  258,  260,  263,
         272,  274,  275,  341,  342,  343,  344,  345,  346,  347,  349,
         350,  351,  352,  353,  370,  371,  372,  373,  374,  375,  376,
         378,  380,  390,  398,  399,  400,  401,  402,  403,  404,  407,
         409,  410,  419,  427,  428,  429,  430,  431,  435,  447,  457,
         458,  459,  460,  461,  462,  463,  465,  466,  467,  468,  469,
         478,  486,  487,  488,  489,  490,  491,  492,  494,  496,  497,
         505,  506,  514,  515,  516,  517,  518,  519,  520,  523,  524,
         526,  543,  544,  547,  549,  562,  563,  587,  633,  665,  666,
         667,  668,  669,  670,  671,  672,  673,  674,  675,  676,  677,
         678,  679,  680,  681,  682,  683,  684,  685,  686,  687,  688,
         689,  690,  691,  692,  693,  694,  695,  696,  697,  698,  699,
         700,  701,  702,  703,  704,  705,  706,  707,  708,  709,  710,
         725,  792,  827,  861,  962,  963,  964,  969,  974,  975,  976,
         977,  981,  982,  986,  987,  988,  989,  993,  994,  998, 1005,
        1010, 1011, 1012, 1013, 1016, 1017, 1018, 1019, 1020, 1021, 1022,
        1023, 1024, 1025, 1029, 1030, 1031, 1034, 1035, 1036, 1037, 1041,
        1042, 1046, 1047, 1049, 1053, 1054, 1056, 1059, 1123, 1136, 1141,
        1143, 1146, 1148, 1149, 1150, 1152, 1156, 1158, 1160, 1163, 1164,
        1165, 1166, 1167, 1169, 1170, 1171, 1173, 1174, 1176, 1177, 1178,
        1179, 1182, 1183, 1185, 1186, 1187, 1188, 1189, 1190, 1191, 1194,
        1195, 1198, 1199, 1200, 1201, 1203, 1206, 1207, 1211, 1212, 1213,
        1214, 1215, 1216, 1217, 1218, 1220, 1221, 1222, 1223, 1226, 1231,
        1236, 1237, 1238, 1239, 1241, 1243, 1244, 1245, 1246, 1247, 1249,
        1250, 1251, 1252, 1253, 1256, 1257, 1258, 1259, 1260, 1261]

In [5]:
id1 = [i for i in id_f if i < 169]
id2 = [i for i in id_f if i >= 169 and i < (169+404)]
id3 = [i for i in id_f if i >= (169+404) and i < (169+404+378)]
id4 = [i for i in id_f if i >= (169+404+378) and i < (169+404+378+107)]
id5 = [i for i in id_f if i >= (169+404+378+107) and i < (169+404+378+107+58)]
id6 = [i for i in id_f if i >= (169+404+378+107+58) and i < 1262]
acc1 = (1-len(id1)/169)*100 ; print ( acc1)
acc2 = (1-len(id2)/404)*100 ; print ( acc2)
acc3 = (1-len(id3)/378)*100 ; print ( acc3)
acc4 = (1-len(id4)/107)*100 ; print ( acc4)
acc5 = (1-len(id5)/58)*100 ; print ( acc5)
acc6 = (1-len(id6)/146)*100 ; print ( acc6)

62.130177514792905
67.82178217821782
86.24338624338624
56.07476635514019
98.27586206896551
45.890410958904106
